# 第二章：Transformer架构

本章节将会结合正文当中的Transformer架构的知识，为大家实际动手操作一下，如何应用Transformer架构做一个中英翻译器，并且最终保存为权重共大家进行推导集的实验


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import math
import torch.nn.functional as F
import jieba  
import re  
from collections import defaultdict, Counter
import os
import pandas as pd

## 读取本地数据集

数据集为english.en（英文句子）和chinese.zh（对应中文翻译），每行一条数据，需保证两条文件的句子一一对应：

In [5]:
def load_data(chinese_path, english_path, max_samples=10000):
    """加载中文和英文数据，最多加载max_samples条"""
    with open(chinese_path, 'r', encoding='utf-8') as f:
        chinese_data = f.readlines()  
    with open(english_path, 'r', encoding='utf-8') as f:
        english_data = f.readlines()
    
    # 确保两种语言数据量一致，并限制数量
    min_len = min(len(chinese_data), len(english_data), max_samples)
    chinese_data = chinese_data[:min_len]
    english_data = english_data[:min_len]
    
    # 创建DataFrame
    df = pd.DataFrame({
        'chinese': chinese_data,
        'english': english_data
    })
    
    # 去除每行末尾的换行符
    df['chinese'] = df['chinese'].str.strip()
    df['english'] = df['english'].str.strip()
    
    return df

chinese_path = 'chinese.zh'
english_path = 'english.en'
# 加载数据
print("数据描述")
data = load_data(chinese_path, english_path)
print(f"数据集大小: {len(data)}")
print(f"数据集前5行:\n{data.head()}")

数据描述
数据集大小: 10000
数据集前5行:
                                             chinese  \
0                                      1929年还是1989年?   
1  巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正...   
2  一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为...   
3  如今人们的心情却是沉重多了，许多人开始把这次危机与1929年和1931年相比，即使一些国家政...   
4                  目前的趋势是，要么是过度的克制（欧洲），要么是努力的扩展（美国）。   

                                             english  
0                                      1929 or 1989?  
1  PARIS – As the economic crisis deepens and wid...  
2  At the start of the crisis, many people likene...  
3  Today, the mood is much grimmer, with referenc...  
4  The tendency is either excessive restraint (Eu...  


## 分词处理

**1.** 中文分词：用jieba实现，对应文档 1.3.1 节 “中文分词”（解决中文无空格分隔问题）。

**2.** 英文分词：用正则表达式分割单词和标点，对应文档 1.3.2 节 “子词切分”（将文本拆分为最小语义单位）。

In [6]:
def tokenize_zh(text):
    """中文分词：保留标点，返回token列表"""
    return list(jieba.cut(text))  # jieba.cut返回生成器，转为列表

def tokenize_en(text):
    """英文分词：用正则分割单词和标点，转为小写"""
    # 匹配字母、数字、 apostrophe（如don't）和标点
    pattern = re.compile(r"[a-zA-Z0-9']+|[^\w\s]")
    tokens = pattern.findall(text.lower())  # 小写统一格式
    return tokens

# 示例：测试分词效果
sample = data.iloc[0]
print(f"中文原句：{sample['chinese']}")
print(f"中文分词：{tokenize_zh(sample['chinese'])}")
print(f"英文原句：{sample['english']}")
print(f"英文分词：{tokenize_en(sample['english'])}")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\xnlll\AppData\Local\Temp\jieba.cache


中文原句：1929年还是1989年?


Loading model cost 0.474 seconds.
Prefix dict has been built successfully.


中文分词：['1929', '年', '还是', '1989', '年', '?']
英文原句：1929 or 1989?
英文分词：['1929', 'or', '1989', '?']


## 构建词汇表

对应文档中的“Embedding 层”内容

词汇表是将 token 映射为索引的关键，用于 Embedding 层查找词向量。手动实现词汇表类

In [7]:
class Vocab:
    def __init__(self, tokens_list, min_freq=2, specials=['<unk>', '<pad>', '<bos>', '<eos>']):
        """
        tokens_list: 所有句子的分词结果列表（如[[token1, token2], ...]）
        min_freq: 过滤低频词（出现次数<min_freq的token视为<unk>）
        specials: 特殊符号（未知词、填充、句首、句尾）
        """
        # 统计token频率
        token_counts = Counter([token for seq in tokens_list for token in seq])
        # 筛选高频词（保留出现次数≥min_freq的token）
        self.token_list = [token for token, cnt in token_counts.items() if cnt >= min_freq]
        # 加入特殊符号（放在最前面，保证索引固定）
        self.token_list = specials + self.token_list
        # 构建token→索引映射
        self.token2idx = {token: idx for idx, token in enumerate(self.token_list)}
        # 构建索引→token映射
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
    
    def __len__(self):
        return len(self.token_list)
    
    def convert_tokens_to_ids(self, tokens):
        """将token列表转为索引列表（未知token用<unk>的索引）"""
        return [self.token2idx.get(token, self.token2idx['<unk>']) for token in tokens]
    
    def convert_ids_to_tokens(self, ids):
        """将索引列表转为token列表"""
        return [self.idx2token[idx] for idx in ids]

# 生成所有句子的分词结果，用于构建词汇表
zh_tokens_all = [tokenize_zh(text) for text in data['chinese']]  # 所有中文句子的分词结果
en_tokens_all = [tokenize_en(text) for text in data['english']]  # 所有英文句子的分词结果

# 构建中、英文词汇表
zh_vocab = Vocab(zh_tokens_all, min_freq=2)  # 中文词汇表
en_vocab = Vocab(en_tokens_all, min_freq=2)  # 英文词汇表

print(f"中文词汇表大小：{len(zh_vocab)}，英文词汇表大小：{len(en_vocab)}")
print(f"中文特殊符号索引：<bos>{zh_vocab.token2idx['<bos>']}，<eos>{zh_vocab.token2idx['<eos>']}，<pad>{zh_vocab.token2idx['<pad>']}")

中文词汇表大小：9821，英文词汇表大小：9082
中文特殊符号索引：<bos>2，<eos>3，<pad>1


## 构建数据集与数据加载器

对应文档 “序列数据处理”

将文本转为模型可输入的张量，添加<bos>（句首）和<eos>（句尾）符号，并对齐序列长度（用<pad>填充）。

In [8]:
class TranslationDataset(Dataset):
    def __init__(self, df, src_vocab, tgt_vocab, src_col='english', tgt_col='chinese'):
        """
        df: 包含源语言和目标语言的DataFrame
        src_vocab: 源语言（英文）词汇表
        tgt_vocab: 目标语言（中文）词汇表
        src_col/tgt_col: DataFrame中源/目标语言列名
        """
        self.df = df
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.src_col = src_col
        self.tgt_col = tgt_col
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # 获取句子
        src_text = self.df.iloc[idx][self.src_col]
        tgt_text = self.df.iloc[idx][self.tgt_col]
        
        # 分词
        src_tokens = tokenize_en(src_text)  # 英文分词
        tgt_tokens = tokenize_zh(tgt_text)  # 中文分词
        
        # 添加句首（<bos>）和句尾（<eos>）符号
        src_tokens = ['<bos>'] + src_tokens + ['<eos>']
        tgt_tokens = ['<bos>'] + tgt_tokens + ['<eos>']
        
        # 转为索引
        src_ids = self.src_vocab.convert_tokens_to_ids(src_tokens)
        tgt_ids = self.tgt_vocab.convert_tokens_to_ids(tgt_tokens)
        
        return torch.tensor(src_ids, dtype=torch.long), torch.tensor(tgt_ids, dtype=torch.long)

# 定义collate_fn：将批次内序列填充至同一长度
def collate_fn(batch, pad_idx_src, pad_idx_tgt):
    """
    batch: 数据集返回的(src_ids, tgt_ids)列表
    pad_idx_src: 源语言<pad>的索引
    pad_idx_tgt: 目标语言<pad>的索引
    """
    src_batch, tgt_batch = zip(*batch)
    # 填充源语言序列（batch_first=True表示输出形状为[batch_size, seq_len]）
    src_padded = pad_sequence(src_batch, batch_first=True, padding_value=pad_idx_src)
    # 填充目标语言序列
    tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=pad_idx_tgt)
    return src_padded, tgt_padded

# 初始化数据集
dataset = TranslationDataset(
    df=data,
    src_vocab=en_vocab,
    tgt_vocab=zh_vocab
)

# 初始化数据加载器
batch_size = 32
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda x: collate_fn(
        x,
        pad_idx_src=en_vocab.token2idx['<pad>'],
        pad_idx_tgt=zh_vocab.token2idx['<pad>']
    )
)

# 测试数据加载器
src_sample, tgt_sample = next(iter(dataloader))
print(f"源语言批次形状：{src_sample.shape}，目标语言批次形状：{tgt_sample.shape}")

源语言批次形状：torch.Size([32, 57])，目标语言批次形状：torch.Size([32, 53])


## 构建 Transformer 模型（核心）

### 1.位置编码

注意力机制本身不包含位置信息，需通过正余弦函数注入序列顺序

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_len=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        # 初始化位置编码矩阵（max_seq_len, d_model）
        pe = torch.zeros(max_seq_len, d_model)
        # 位置索引（0到max_seq_len-1）
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        # 计算频率因子（避免数值过大）
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        # 偶数维度用正弦，奇数维度用余弦
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        # 增加batch维度（1, max_seq_len, d_model）
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)  # 不参与训练的缓冲区
    
    def forward(self, x):
        """x: 输入序列嵌入，形状为[batch_size, seq_len, d_model]"""
        x = x + self.pe[:, :x.size(1)]  # 加位置编码（自动广播batch维度）
        return self.dropout(x)

### 注意力机制

实现核心公式 

$$
attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V 
$$

In [10]:
def attention(query, key, value, mask=None, dropout=None):
    """
    query: [batch_size, n_heads, seq_len_q, d_k]
    key: [batch_size, n_heads, seq_len_k, d_k]
    value: [batch_size, n_heads, seq_len_v, d_v]
    mask: [batch_size, 1, seq_len_q, seq_len_k]（1表示可见，0表示遮蔽）
    """
    d_k = query.size(-1)
    # 计算相似度分数：Q*K^T / sqrt(d_k)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    # 应用掩码（遮蔽<pad>或未来信息）
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)  # 掩码位置设为负无穷
    # softmax归一化得到注意力权重
    attn_weights = F.softmax(scores, dim=-1)
    # 应用dropout
    if dropout is not None:
        attn_weights = dropout(attn_weights)
    # 加权求和得到输出
    output = torch.matmul(attn_weights, value)
    return output, attn_weights

### 多头注意力

将注意力拆分为多个头并行计算，捕捉不同语义关系

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        assert d_model % n_heads == 0, "d_model必须是n_heads的整数倍"
        self.d_k = d_model // n_heads  # 每个头的维度
        self.n_heads = n_heads
        # Q/K/V的线性变换矩阵（将d_model映射到d_model）
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        # 输出线性变换矩阵（将多头结果拼接后映射回d_model）
        self.w_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        """
        query/key/value: [batch_size, seq_len, d_model]
        """
        batch_size = query.size(0)
        # 线性变换并拆分多头：[batch_size, seq_len, d_model] → [batch_size, n_heads, seq_len, d_k]
        q = self.w_q(query).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = self.w_k(key).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        v = self.w_v(value).view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        # 计算注意力
        output, attn_weights = attention(q, k, v, mask, self.dropout)
        # 拼接多头结果：[batch_size, n_heads, seq_len, d_k] → [batch_size, seq_len, d_model]
        output = output.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_k)
        # 输出线性变换
        return self.w_o(output), attn_weights

### 前馈网络与层归一化

- 前馈网络（FFN）：增强模型非线性能力，对每个位置独立处理。

- 层归一化 + 残差连接：稳定训练，缓解深层模型梯度问题。

In [12]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)  # 升维
        self.fc2 = nn.Linear(d_ff, d_model)  # 降维
        self.dropout = nn.Dropout(dropout)
        self.activation = F.relu  # 非线性激活
    
    def forward(self, x):
        return self.fc2(self.dropout(self.activation(self.fc1(x))))

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.gamma = nn.Parameter(torch.ones(features))  # 缩放参数
        self.beta = nn.Parameter(torch.zeros(features))   # 偏移参数
        self.eps = eps  # 防止除零
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)  # 最后一维求均值
        std = x.std(-1, keepdim=True)    # 最后一维求标准差
        return self.gamma * (x - mean) / (std + self.eps) + self.beta

### Encoder 与 Decoder 层

- Encoder 层：由 “多头自注意力 + 前馈网络” 组成，输出源语言编码。

- Decoder 层：由 “掩码多头自注意力（遮蔽未来信息）+ 编码器 - 解码器注意力（关联源 - 目标语言）+ 前馈网络” 组成。

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)  # 自注意力
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff, dropout)  # 前馈网络
        self.norm1 = LayerNorm(d_model)  # 层归一化1
        self.norm2 = LayerNorm(d_model)  # 层归一化2
        self.dropout1 = nn.Dropout(dropout)  #  dropout1
        self.dropout2 = nn.Dropout(dropout)  #  dropout2
    
    def forward(self, x, mask):
        # 自注意力 + 残差连接 + 层归一化
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout1(attn_output))
        # 前馈网络 + 残差连接 + 层归一化
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout2(ff_output))
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, n_heads, dropout)  # 掩码自注意力
        self.cross_attn = MultiHeadAttention(d_model, n_heads, dropout)  # 编码器-解码器注意力
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff, dropout)  # 前馈网络
        self.norm1 = LayerNorm(d_model)  # 层归一化1
        self.norm2 = LayerNorm(d_model)  # 层归一化2
        self.norm3 = LayerNorm(d_model)  # 层归一化3
        self.dropout1 = nn.Dropout(dropout)  #  dropout1
        self.dropout2 = nn.Dropout(dropout)  #  dropout2
        self.dropout3 = nn.Dropout(dropout)  #  dropout3
    
    def forward(self, x, enc_output, self_mask, cross_mask):
        # 掩码自注意力（遮蔽未来信息）
        attn_output, _ = self.self_attn(x, x, x, self_mask)
        x = self.norm1(x + self.dropout1(attn_output))
        # 编码器-解码器注意力（关联源语言和目标语言）
        attn_output, _ = self.cross_attn(x, enc_output, enc_output, cross_mask)
        x = self.norm2(x + self.dropout2(attn_output))
        # 前馈网络
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout3(ff_output))
        return x

### 组装成完整 Transformer 模型

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, n_layers=6, 
                 n_heads=8, d_ff=2048, max_seq_len=5000, dropout=0.1):
        super().__init__()
        # 编码器
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)  # 源语言Embedding
        self.encoder_pos_encoding = PositionalEncoding(d_model, max_seq_len, dropout)
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)
        ])
        # 解码器
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)  # 目标语言Embedding
        self.decoder_pos_encoding = PositionalEncoding(d_model, max_seq_len, dropout)
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)
        ])
        # 输出层（映射到目标语言词汇表）
        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.d_model = d_model  # 模型维度
    
    def generate_masks(self, src, tgt):
        """生成编码器和解码器掩码"""
        batch_size, src_len = src.shape
        batch_size, tgt_len = tgt.shape
        
        # 编码器掩码：遮蔽<pad>（src中<pad>的位置设为0）
        src_mask = (src != en_vocab.token2idx['<pad>']).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, src_len]
        
        # 解码器自注意力掩码：遮蔽<pad>和未来信息（下三角矩阵）
        tgt_self_mask = (tgt != zh_vocab.token2idx['<pad>']).unsqueeze(1).unsqueeze(2)  # [batch, 1, 1, tgt_len]
        
        # 确保下三角矩阵为布尔类型
        tgt_tri_mask = torch.tril(torch.ones(tgt_len, tgt_len, device=tgt.device, dtype=torch.bool))
        
        # 确保两个掩码都是布尔类型再进行位运算
        tgt_self_mask = tgt_self_mask & tgt_tri_mask  # 结合两种掩码
        
        # 解码器交叉注意力掩码：与编码器掩码一致（确保对齐源语言）
        tgt_cross_mask = src_mask
        
        return src_mask, tgt_self_mask, tgt_cross_mask
    
    def forward(self, src, tgt):
        """
        src: 源语言序列（英文），形状[batch_size, src_len]
        tgt: 目标语言序列（中文），形状[batch_size, tgt_len]（训练时不含最后一个<eos>）
        """
        # 生成掩码
        src_mask, tgt_self_mask, tgt_cross_mask = self.generate_masks(src, tgt)
        
        # 编码器前向
        enc_emb = self.encoder_embedding(src) * math.sqrt(self.d_model)  # Embedding缩放
        enc_emb = self.encoder_pos_encoding(enc_emb)  # 加位置编码
        enc_output = enc_emb
        for layer in self.encoder_layers:
            enc_output = layer(enc_output, src_mask)  
        
        # 解码器前向
        dec_emb = self.decoder_embedding(tgt) * math.sqrt(self.d_model)  # Embedding缩放
        dec_emb = self.decoder_pos_encoding(dec_emb)  # 加位置编码
        dec_output = dec_emb
        for layer in self.decoder_layers:
            dec_output = layer(dec_output, enc_output, tgt_self_mask, tgt_cross_mask)  # 经过所有解码器层
        
        # 输出层（映射到目标词汇表）
        output = self.fc(dec_output)
        return output

## 模型训练与训练集保存

In [15]:
# 超参数
d_model = 512  # 模型维度（原论文512）
n_layers = 6  # 编码器/解码器层数（原论文6）
n_heads = 8   # 注意力头数（原论文8）
d_ff = 2048   # 前馈网络隐藏层维度（原论文2048）
dropout = 0.1
epochs = 30   # 训练轮数
lr = 0.0001   # 学习率

# 初始化模型
model = Transformer(
    src_vocab_size=len(en_vocab),
    tgt_vocab_size=len(zh_vocab),
    d_model=d_model,
    n_layers=n_layers,
    n_heads=n_heads,
    d_ff=d_ff,
    dropout=dropout
)

# 损失函数（忽略<pad>的损失）
criterion = nn.CrossEntropyLoss(ignore_index=zh_vocab.token2idx['<pad>'])

# 优化器（Adam）
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.98), eps=1e-9)

准备完成之后开始训练我们自己的模型啦！

In [13]:
# 设备选择（GPU优先）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion.to(device)

# 训练循环
for epoch in range(epochs):
    model.train()  # 训练模式
    total_loss = 0.0
    for batch_idx, (src, tgt) in enumerate(dataloader):
        # 移动数据到设备
        src = src.to(device)  # [batch_size, src_len]
        tgt = tgt.to(device)  # [batch_size, tgt_len]
        
        # 解码器输入：tgt[:, :-1]（去除最后一个<eos>）
        # 解码器目标：tgt[:, 1:]（去除第一个<bos>）
        output = model(src, tgt[:, :-1])  # [batch_size, tgt_len-1, tgt_vocab_size]
        
        # 计算损失（CrossEntropyLoss要求输入形状为[batch, vocab_size, seq_len]）
        loss = criterion(output.transpose(1, 2), tgt[:, 1:])
        
        # 反向传播
        optimizer.zero_grad()  # 清零梯度
        loss.backward()        # 计算梯度
        optimizer.step()       # 更新参数
        
        total_loss += loss.item()
    
    # 打印轮次平均损失
    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1} 平均损失: {avg_loss:.4f}\n")

# 保存模型权重
torch.save(model.state_dict(), 'transformer_zh_en.pth')
print("模型权重已保存至 'transformer_zh_en.pth'")

Epoch 1 平均损失: 6.5164

Epoch 2 平均损失: 5.9108

Epoch 3 平均损失: 5.5808

Epoch 4 平均损失: 5.2947

Epoch 5 平均损失: 5.0408

Epoch 6 平均损失: 4.8072

Epoch 7 平均损失: 4.5925

Epoch 8 平均损失: 4.3832

Epoch 9 平均损失: 4.1815

Epoch 10 平均损失: 3.9823

Epoch 11 平均损失: 3.7903

Epoch 12 平均损失: 3.5957

Epoch 13 平均损失: 3.4023

Epoch 14 平均损失: 3.2126

Epoch 15 平均损失: 3.0264

Epoch 16 平均损失: 2.8420

Epoch 17 平均损失: 2.6566

Epoch 18 平均损失: 2.4752

Epoch 19 平均损失: 2.2976

Epoch 20 平均损失: 2.1292

Epoch 21 平均损失: 1.9595

Epoch 22 平均损失: 1.8008

Epoch 23 平均损失: 1.6442

Epoch 24 平均损失: 1.4992

Epoch 25 平均损失: 1.3558

Epoch 26 平均损失: 1.2265

Epoch 27 平均损失: 1.1065

Epoch 28 平均损失: 0.9931

Epoch 29 平均损失: 0.8851

Epoch 30 平均损失: 0.7962

模型权重已保存至 'transformer_zh_en.pth'


## 使用预训练集进行推导

In [ ]:
def preprocess_sentence(sentence, vocab, tokenize_fn):
    """预处理输入句子，转为模型可接受的格式"""
    tokens = tokenize_fn(sentence)  # 分词
    tokens = ['<bos>'] + tokens + ['<eos>']  # 添加句首/句尾符号
    indices = vocab.convert_tokens_to_ids(tokens)  # 转为索引
    tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0)  # 添加batch维度
    return tensor

def translate_sentence(model, src_sentence, src_vocab, tgt_vocab, 
                       src_tokenize_fn, tgt_tokenize_fn, device, max_len=50):
    """将英文句子翻译成中文"""
    model.eval()  # 评估模式，关闭dropout等训练特有的层
    with torch.no_grad():  # 关闭梯度计算，节省内存
        # 预处理源语言句子
        src_tensor = preprocess_sentence(src_sentence, src_vocab, src_tokenize_fn).to(device)
        
        # 编码器处理源语言
        src_mask = (src_tensor != src_vocab.token2idx['<pad>']).unsqueeze(1).unsqueeze(2)
        enc_emb = model.encoder_embedding(src_tensor) * math.sqrt(model.d_model)
        enc_emb = model.encoder_pos_encoding(enc_emb)
        enc_output = enc_emb
        for layer in model.encoder_layers:
            enc_output = layer(enc_output, src_mask)
        
        # 解码器自回归生成目标句子（从<bos>开始）
        tgt_indices = [tgt_vocab.token2idx['<bos>']]
        
        for _ in range(max_len):
            tgt_tensor = torch.tensor(tgt_indices, dtype=torch.long).unsqueeze(0).to(device)
            
            # 生成解码器掩码（遮蔽未来信息和<pad>）
            tgt_self_mask = (tgt_tensor != tgt_vocab.token2idx['<pad>']).unsqueeze(1).unsqueeze(2)
            tgt_tri_mask = torch.tril(
                torch.ones(tgt_tensor.size(1), tgt_tensor.size(1), device=device, dtype=torch.bool)
            )
            tgt_self_mask = tgt_self_mask & tgt_tri_mask
            tgt_cross_mask = src_mask  # 与编码器掩码一致
            
            # 解码器前向计算
            dec_emb = model.decoder_embedding(tgt_tensor) * math.sqrt(model.d_model)
            dec_emb = model.decoder_pos_encoding(dec_emb)
            dec_output = dec_emb
            for layer in model.decoder_layers:
                dec_output = layer(dec_output, enc_output, tgt_self_mask, tgt_cross_mask)
            output = model.fc(dec_output)

            # 取最后一个位置的预测结果
            next_token_idx = output[:, -1, :].argmax(1).item()
            tgt_indices.append(next_token_idx)
            
            # 遇到<eos>停止生成
            if next_token_idx == tgt_vocab.token2idx['<eos>']:
                break
        
        # 转换为中文句子（移除特殊符号）
        tgt_tokens = tgt_vocab.convert_ids_to_tokens(tgt_indices)
        translated_sentence = ''.join(tgt_tokens[1:-1])  # 跳过<bos>和<eos>
        return translated_sentence

# 加载模型（需与训练时的超参数一致）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
d_model = 512
n_layers = 6
n_heads = 8
d_ff = 2048
dropout = 0.1

# 初始化模型并加载权重
model = Transformer(
    src_vocab_size=len(en_vocab),
    tgt_vocab_size=len(zh_vocab),
    d_model=d_model,
    n_layers=n_layers,
    n_heads=n_heads,
    d_ff=d_ff,
    dropout=dropout
).to(device)
model.load_state_dict(torch.load('transformer_zh_en.pth', map_location=device))

# 交互翻译
print("===== 中英文翻译器 =====")
print("提示：输入英文句子进行翻译，输入'q'退出")
while True:
    user_input = input("\n请输入英文句子: ")
    if user_input.lower() == 'q':
        print("程序已退出")
        break
    # 执行翻译
    translation = translate_sentence(
        model=model,
        src_sentence=user_input,
        src_vocab=en_vocab,
        tgt_vocab=zh_vocab,
        src_tokenize_fn=tokenize_en,  # 英文分词函数
        tgt_tokenize_fn=tokenize_zh,  # 中文分词函数
        device=device
    )
    print(f"翻译结果: {translation}")

===== 中英文翻译器 =====
提示：输入英文句子进行翻译，输入'q'退出
翻译结果: 巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望我们了解目前正在发生的情况。
翻译结果: 但如果这样的情况发生在危机前<unk>，那么有许多国家的人真正稳定自己的那么重要，那就是自己的自己的自己国家，就会有自己的自己的自己的吗？
程序已退出
